In [1]:
import findspark
findspark.init("/usr/hdp/current/spark2-client/")

from pyspark.sql import SparkSession

spark = SparkSession.builder \
.appName("DataframeToDisk_YARN") \
.master("yarn") \
.getOrCreate()

In [2]:
from pyspark.sql.types import *

schema = StructType([
    
StructField("sirano", IntegerType(), True),
StructField("isim", StringType(), True),
StructField("yas", IntegerType(), True),
StructField("meslek", StringType(), True),
StructField("sehir", StringType(), True),
StructField("aylik_gelir", DoubleType(), True)

])

In [3]:
df = spark.read.format("csv") \
.option("header", True) \
.option("sep", ",") \
.schema(schema) \
.load("/user/erkan/simple_data.csv")

In [4]:
from pyspark.sql import functions as f
df2 = df.withColumn("meslek_sehir", f.concat(f.col("meslek"), f.lit(" - "), f.col("sehir")))
df2.show()

+------+--------+---+-----------+--------+-----------+--------------------+
|sirano|    isim|yas|     meslek|   sehir|aylik_gelir|        meslek_sehir|
+------+--------+---+-----------+--------+-----------+--------------------+
|     1|   Cemal| 35|       Isci|  Ankara|     3500.0|       Isci - Ankara|
|     2|   Ceyda| 42|      Memur| Kayseri|     4200.0|     Memur - Kayseri|
|     3|   Timur| 30|   Müzisyen|Istanbul|     9000.0| Müzisyen - Istanbul|
|     4|   Burcu| 29|Pazarlamaci|  Ankara|     4200.0|Pazarlamaci - Ankara|
|     5| Yasemin| 23|Pazarlamaci|   Bursa|     4800.0| Pazarlamaci - Bursa|
|     6|     Ali| 33|      Memur|  Ankara|     4250.0|      Memur - Ankara|
|     7|   Dilek| 29|Pazarlamaci|Istanbul|     7300.0|Pazarlamaci - Ist...|
|     8|   Murat| 31|   Müzisyen|Istanbul|    12000.0| Müzisyen - Istanbul|
|     9|   Ahmet| 33|     Doktor|  Ankara|    18000.0|     Doktor - Ankara|
|    10|Muhittin| 46|     Berber|Istanbul|    12000.0|   Berber - Istanbul|
|    11|Hica

In [6]:
df2.coalesce(1) \
.write \
.mode("overwrite") \
.option("sep",",") \
.option("header","true") \
.csv("/user/erkan/simple_data2")